In [847]:
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta
import time
#import pymc3
import math
import arviz as az
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import seaborn as sns

from zipfile import ZipFile
import scipy as sp


import sys, subprocess, glob, os, shutil, re, importlib
from subprocess import call
import imp


%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import matplotlib.colors as clr
from matplotlib import rc
import textwrap as textwrap
from textwrap import wrap

import warnings
warnings.filterwarnings('ignore')

from scipy.special import binom

In [848]:
def read_in_Ne_changes_mascot(log_file_path):
    
    Ne_skyline_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                #print(line)
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "Ne." in col:
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        #deme = name.split(".")[1]# the syntax here is "Ne_region.1" where region is deme and 1 is interval 1
                        interval = name.split(".")[1]
                       
                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])
                    
                
    return(Ne_skyline_dict)


In [849]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "Ne." in i:
            #deme = i.split(".")[1]
           # print(deme)
            #interval = 
            #print(interval)
#             if "\n" in i.split(".")[2]:
#                 interval = i.split(".")[2][0:2]
#             else:
            interval = i.split(".")[1]
           # print(interval)
            #print(interval)
            #print(i)
            #next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = math.exp(mean_log)
            median_linear = math.exp(median_log)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
            
#             try:
#                 next_local_series = input_df["SkylineNe"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
#                 diff_series = np.subtract(local_series, next_local_series)
#                 #print(local_series)
#                 #print(next_local_series)
#                 #print(diff_series)
#                 diff_mean_log = diff_series.mean()
#                 diff_median_log = np.median(diff_series)
#                 diff_hpd_95 = az.hdi(diff_series, 0.95)
#                 diff_lower_hpd_log_95 = diff_hpd_95[0]
#                 diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
#                 diff_upper_hpd_log_95 = diff_hpd_95[1]
#                 diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
#                 diff_hpd_50 = az.hdi(diff_series, 0.50)
#                 diff_lower_hpd_log_50 = diff_hpd_50[0]
#                 diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
#                 diff_upper_hpd_log_50 = diff_hpd_50[1]
#                 diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
#             except KeyError:
#                 pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "median_Ne_log" : median_log, "median_Ne_linear": median_linear, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                  })
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
    return(new_df)

In [850]:
#calculating transmission rate
def generate_summary_diff_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne." in i:
            #deme = i.split(".")[1]
           # print(deme)
            #interval = 
            #print(interval)
#             if "\n" in i.split(".")[2]:
#                 interval = i.split(".")[2][0:2]
#             else:
            interval = i.split(".")[1]
            next_interval = int(interval)+4 #averaging over two weeks to reduce noise
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
           
            try:
                new_df["Ne"+ ".diff." + str(interval)] = (52/4)*(input_df[i].astype("float") - input_df["Ne"+"." + str(next_interval)].astype('float'))
            
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [851]:
def generate_local_and_intro_rt(input_df):
    
    
    new_df = pd.DataFrame()
    incubation_period = 365/14
    uninfectious_rate = 365/4.5
    
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[2]
            new_df["rt" +  "." + str(interval)] = (1+ (input_df[i].astype("float") / uninfectious_rate)) * (1+ (input_df[i].astype("float") / incubation_period))
            

            
    return(new_df)

In [852]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_rt_summary_df(input_df):
    
    
    new_df = pd.DataFrame()
    count = 0
    for i in input_df.columns.tolist():
        #print(i)
        if "rt" in i:
            if count %1 == 0:
                interval = i.split(".")[1]
                #interval = i.split(".")[2]
                local_series = input_df[i].astype('float').to_numpy()
                mean_percent = local_series.mean()
                median_percent = np.median(local_series)

                hpd_95 = az.hdi(local_series, 0.95)
                lower_hpd_log_95 = hpd_95[0]
                upper_hpd_log_95 = hpd_95[1]
                hpd_50 = az.hdi(local_series, 0.50)
                lower_hpd_log_50 = hpd_50[0]
                upper_hpd_log_50 = hpd_50[1]

                try:
                    local_df = pd.DataFrame.from_dict({"interval":interval, "mean_percent":mean_percent, "median_percent":median_percent,
                                                       "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                       "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                    new_df = new_df.append(local_df)
                except:
                    pass
            count+=1

    return(new_df)

In [853]:

log_file_path = "../simulations/beast_results/updated_simulation_multicoal_case_prior_4_all_rolling.log"

In [854]:
#read in log file
Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

#remove burn-in
Ne_df = pd.DataFrame.from_dict(Ne_skyline)
print(len(Ne_df))

burnin_percent = 0.3
print(len(Ne_df))
rows_to_remove = int(len(Ne_df)* burnin_percent)
Ne_df = Ne_df.iloc[rows_to_remove:]

print(len(Ne_df))
Ne_df = Ne_df.reset_index()

ne_summary = generate_summary_df(Ne_df)

#format time intervals
test = ne_summary
test['days'] = (test.interval.astype(int))*7
test['date'] = dt.strptime("2023-01-14",  "%Y-%m-%d") - test.days.map(timedelta)

test["new_interval"] = (((test.interval.iloc[::-1]).astype(int))*7 )-7

#add smoothing 
columns = ["mean_Ne_log","mean_Ne_linear",  "median_Ne_linear","median_Ne_log", "upper_hpd_log_50","lower_hpd_log_50", "upper_hpd_linear_50","lower_hpd_linear_50", "lower_hpd_linear", "upper_hpd_linear"]
for column in columns:
    # Create a new column for the moving average
    test[f'{column}_MA'] = test[column].rolling(5, min_periods =1).mean()


5001
5001
3501


In [855]:
#plot
range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]
log_line_ne = alt.Chart(test).mark_area(interpolate='monotone', opacity = 0.8, clip = True).encode(
    alt.X('new_interval', ),
    alt.Y('upper_hpd_log_50_MA',axis=alt.Axis(title="Effective Population Size", grid=False), scale = alt.Scale(domain=[-3,8])),
    alt.Y2('lower_hpd_log_50_MA' ),
    
).properties(
    width=850,
    height=300
)
log_band_ne = alt.Chart(test).mark_line(interpolate='monotone', opacity = 0.5, clip = True, color = "black").encode(
    alt.X('new_interval', ),
    alt.Y('median_Ne_log_MA',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),

    
).properties(
    width=850,
    height=300
)

log_line_ne  + log_band_ne

alt.LayerChart(...)

In [856]:
ne_diff_summary = generate_summary_diff_df(Ne_df)

uninfectious_rate = 365/4.5

#taken from https://www.medrxiv.org/content/10.1101/2022.08.17.22278897v1.full.pdf
incubation_period = 365/8

seir_transmission_rate = ((ne_diff_summary*2 + uninfectious_rate + incubation_period)**2 - (incubation_period- uninfectious_rate)**2)/(4*incubation_period)
seir_transmission_rate.head()

,Ne.diff.1,Ne.diff.2,Ne.diff.3,Ne.diff.4,Ne.diff.5,Ne.diff.6,Ne.diff.7,Ne.diff.8,Ne.diff.9,Ne.diff.10,...,Ne.diff.25,Ne.diff.26,Ne.diff.27,Ne.diff.28,Ne.diff.29,Ne.diff.30,Ne.diff.31,Ne.diff.32,Ne.diff.33,Ne.diff.34
0,196.780514,26.058036,-5.044777,79.859604,370.026064,132.590351,67.036774,154.419650,83.504662,69.468518,...,-6.538452,428.424199,169.721260,341.374728,259.713013,51.792970,362.671450,287.192893,429.390714,567.317431
1,92.450972,16.466335,-1.015760,40.725642,278.063746,123.981413,117.284519,110.667415,71.375609,95.390551,...,1.744172,229.817198,355.458831,372.375744,193.872442,166.581664,214.729132,389.239859,1009.853237,1226.261577
2,203.310448,17.769470,-0.626374,166.969938,328.577581,142.154834,70.009142,115.916319,73.247001,89.832061,...,1.157911,72.597271,301.023684,322.786417,161.027792,156.369973,179.558772,405.088778,1467.086233,2342.831482
3,56.040681,-1.305797,-0.310091,36.426851,426.270542,108.571031,82.275541,109.276975,62.225347,90.336475,...,-6.796498,33.638951,220.758568,165.032843,252.146790,321.964270,183.660862,236.064253,848.494551,1102.467015
4,229.027443,5.399534,-3.767087,60.127193,411.569929,117.900175,74.907499,134.215553,108.541406,73.421788,...,25.870617,139.213206,226.082063,922.412623,449.716052,305.763320,149.291956,246.808305,1329.503467,1147.247805


In [899]:
simulation_df = pd.read_json("../simulations/simulation_results/eir_4.json", orient = "index")
simulation_df = simulation_df.T
simulation_df.I[simulation_df.I < 0.1] = 0.1
simulation_df.I = simulation_df.I.astype("float")
simulation_df["log_I"] = simulation_df.I.apply(np.log)
simulation_df

,R,samp2,samp,t,E,IntroCount,sim,I,Iinit,log_I
0,0.0,0.0,0.0,0.0,0.0,0.0,populationEndConditions,0.1,1.0,-2.302585
1,0.0,0.0,0.0,0.001677,0.0,1.0,postSimConditions,1.0,1.0,0.000000
2,0.0,0.0,0.0,0.002968,0.0,2.0,stepper,2.0,1.0,0.693147
3,0.0,0.0,0.0,0.007245,3.0,2.0,lineageEndConditions,1.0,1.0,0.000000
4,0.0,0.0,0.0,0.007372,3.0,3.0,leafCountEndConditions,2.0,1.0,0.693147
...,...,...,...,...,...,...,...,...,...,...
3391,830.0,0.0,516.0,0.702648,22.0,240.0,NaN,5.0,1.0,1.609438
3392,831.0,0.0,516.0,0.702734,22.0,240.0,NaN,4.0,1.0,1.386294
3393,831.0,0.0,516.0,0.703367,21.0,240.0,NaN,5.0,1.0,1.609438
3394,831.0,0.0,516.0,0.70479,21.0,241.0,NaN,6.0,1.0,1.791759


In [900]:
simulation_df["new_time"] = simulation_df.t *366
line_simulation = alt.Chart(simulation_df).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "orange").encode(
    alt.X('new_time', axis=alt.Axis(title="")),
    alt.Y('log_I',axis=alt.Axis(title="Simulated prevalence", grid=False)),
    
    
).properties(
    width=850,
    height=300)
line_simulation

alt.Chart(...)

In [901]:
(line_simulation + log_line_ne + log_band_ne).resolve_scale(x = "shared", y = "shared")

alt.LayerChart(...)

In [902]:
## calculate Rt

In [903]:
rt_local_and_intro_df = generate_local_and_intro_rt(ne_diff_summary)
summary_rt_local_and_intro_df = generate_rt_summary_df(rt_local_and_intro_df)
summary_rt_local_and_intro_df["new_interval"] = (((summary_rt_local_and_intro_df.interval.iloc[::-1]).astype(int))*7 )-7

In [904]:
columns = ["mean_percent","median_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
for column in columns:
    # Create a new column for the moving average
    summary_rt_local_and_intro_df[f'{column}_MA'] = summary_rt_local_and_intro_df[column].rolling(3, min_periods =1, center = True).mean()

In [905]:
summary_rt_local_and_intro_df.head()

,interval,mean_percent,median_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,new_interval,mean_percent_MA,median_percent_MA,upper_hpd_log_95_MA,lower_hpd_log_95_MA,upper_hpd_log_50_MA,lower_hpd_log_50_MA
0,1,2.676730,1.294405,10.537189,-0.358135,1.294405,-0.358135,231,1.601707,0.685958,6.660780,-0.358135,0.685958,-0.358135
0,2,0.526685,0.077511,2.784371,-0.358135,0.077511,-0.358135,224,1.073363,0.385583,4.846977,-0.358135,0.385583,-0.358135
0,3,0.016674,-0.215166,1.219370,-0.358135,-0.215166,-0.358135,217,0.716189,0.192354,3.555432,-0.358135,0.192354,-0.358135
0,4,1.605209,0.714717,6.662555,-0.358135,0.714717,-0.358135,210,3.361282,2.611661,9.015313,0.071908,3.129323,0.859626
0,5,8.461964,7.335432,19.164013,0.931993,8.888417,3.295147,203,3.680184,2.922357,9.507692,0.071928,3.540021,1.007327


In [906]:
# simulation_df_copy = simulation_df.copy()
# simulation_df_copy["diff"] = simulation_df_copy.log_I.diff() *52
# simulation_df_copy["whole_time"] = np.ceil(simulation_df_copy.t *366)



In [940]:
simulation_df_copy = simulation_df.copy()

simulation_df_copy["whole_time"] = np.ceil(simulation_df_copy.t *366)

daily_sim_cases = simulation_df_copy.groupby(["whole_time"]).agg({'log_I':'mean'}).reset_index()


#daily_sim_cases["diff"] = daily_sim_cases.log_I.diff() *52

#daily_sim_cases["diff"][daily_sim_cases["diff"] < 0 ] = 0

#define to week number
counter = 0
daily_sim_cases["week_time"] = np.nan
for index, row in daily_sim_cases.iterrows():
    if row.whole_time %7 == 0:
        daily_sim_cases.loc[index,"week_time"] = counter 
        counter +=1
        
#backfill
daily_sim_cases = daily_sim_cases.bfill()

# aggregate based on weeks
weekly_sim_cases = daily_sim_cases.groupby(["week_time"])["log_I"].sum().reset_index()

weekly_sim_cases["diff"] = weekly_sim_cases.log_I.diff() *7

weekly_sim_cases.head()

,week_time,log_I,diff
0,0.0,-2.302585,NaN
1,1.0,5.976819,57.955826
2,2.0,17.605763,81.402614
3,3.0,25.085271,52.356556
4,4.0,27.498128,16.889997


In [941]:
incubation_period = 365/8
uninfectious_rate = 365/4.5

weekly_sim_cases["Rt"] = (1+ (weekly_sim_cases["diff"] / uninfectious_rate)) * (1+ (weekly_sim_cases["diff"] / incubation_period))
weekly_sim_cases.week_time = weekly_sim_cases.week_time *7

In [942]:
columns = ["Rt"]
for column in columns:
    # Create a new column for the moving average
    weekly_sim_cases[f'{column}_MA'] = weekly_sim_cases[column].rolling(4, min_periods =1).mean()


In [946]:

line_rt_sim = alt.Chart(weekly_sim_cases).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "blue").encode(
    alt.X('week_time', axis=alt.Axis(title="")),
    alt.Y('Rt',axis=alt.Axis(title="Rt", grid=False), scale = alt.Scale(domain = [0.3, 10])),
    
    
).properties(
    width=850,
    height=300)
line_rt_sim

alt.Chart(...)

In [947]:
summary_rt_local_and_intro_df.head()

,interval,mean_percent,median_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,new_interval,mean_percent_MA,median_percent_MA,upper_hpd_log_95_MA,lower_hpd_log_95_MA,upper_hpd_log_50_MA,lower_hpd_log_50_MA
0,1,2.676730,1.294405,10.537189,-0.358135,1.294405,-0.358135,231,1.601707,0.685958,6.660780,-0.358135,0.685958,-0.358135
0,2,0.526685,0.077511,2.784371,-0.358135,0.077511,-0.358135,224,1.073363,0.385583,4.846977,-0.358135,0.385583,-0.358135
0,3,0.016674,-0.215166,1.219370,-0.358135,-0.215166,-0.358135,217,0.716189,0.192354,3.555432,-0.358135,0.192354,-0.358135
0,4,1.605209,0.714717,6.662555,-0.358135,0.714717,-0.358135,210,3.361282,2.611661,9.015313,0.071908,3.129323,0.859626
0,5,8.461964,7.335432,19.164013,0.931993,8.888417,3.295147,203,3.680184,2.922357,9.507692,0.071928,3.540021,1.007327


In [948]:
band_rt = alt.Chart(summary_rt_local_and_intro_df).mark_area(interpolate='monotone', opacity = 0.5, clip = True, color = "orange").encode(
    alt.X('new_interval', axis=alt.Axis(title="")),
    alt.Y('upper_hpd_log_50_MA',axis=alt.Axis(title="Rt", grid=False)),
    alt.Y2("lower_hpd_log_50_MA")
    
    
).properties(
    width=850,
    height=300)

line_rt = alt.Chart(summary_rt_local_and_intro_df).mark_line(interpolate='monotone', opacity = 0.5, clip = True, color = "black").encode(
    alt.X('new_interval', axis=alt.Axis(title="")),
    alt.Y('median_percent_MA',axis=alt.Axis(title="Rt", grid=False)),

    
    
).properties(
    width=850,
    height=300)

(band_rt + line_rt+ line_rt_sim).resolve_scale(y ="shared")

alt.LayerChart(...)

In [870]:
## calculate percentage of cases due to introduction

In [871]:
def read_in_forward_migration_rates_mascot(log_file_path):
    
    mig_rates_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            #print(line_number)
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    mig_column_indices = []   # list to store column indices
                    mig_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "immigrationRate" in col:
                            mig_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in mig_column_indices:
                        name = line.split("\t")[n]
                        interval = name.split(".")[1]# the syntax here is "NeLog.state01" where 0 is deme and 1 is interval 1
                        #interval = name.split(".")[2]
                       
                        mig_key[n] = name
                        mig_rates_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    mig_rates_dict["sample"].append(sample)

                    for index in mig_column_indices:
                        name = mig_key[index]
                        mig_rates_dict[name].append(line.split("\t")[index])
                    
                
                
                
    return(mig_rates_dict)

In [872]:
def generate_percent_intro_df(input_df):
    
    temp_df = pd.DataFrame()
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "immigrationRate" in i:
        
            interval = i.split(".")[1]
            #deme = i.split(".")[2]

            try:
                temp_df["total."+ str(interval)] = seir_transmission_rate["Ne.diff." + str(interval)].astype("float") +  input_df[i].astype("float").apply(np.exp)
                new_df["intro.percent"+ "." + str(interval)] = input_df[i].astype("float").apply(np.exp).div(temp_df["total."+ str(interval)], axis = 0) 

            except KeyError: #this was added because not all regions have equal time periods for their epidemics and it was throwing an error everytime it had to switch deme
                pass 
                  
    return(new_df)

In [873]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "percent" in i:
            interval = i.split(".")[2]
            #interval = i.split(".")[3]
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_percent = np.median(local_series)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]
            

            
            
            try:
                local_df = pd.DataFrame.from_dict({ "interval":interval, "mean_percent":mean_percent, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)


In [874]:
migration_rates_f = read_in_forward_migration_rates_mascot(log_file_path)
mig_df_f = pd.DataFrame.from_dict(migration_rates_f)

#remove burnin
burnin_percent = 0.3
print(len(mig_df_f))
rows_to_remove = int(len(mig_df_f)* burnin_percent)
mig_df_f = mig_df_f.iloc[rows_to_remove:]

print(len(mig_df_f))
mig_df_f = mig_df_f.reset_index()
mig_df_f.head()

5001
3501


,index,sample,immigrationRate.1,immigrationRate.2,immigrationRate.3,immigrationRate.4,immigrationRate.5,immigrationRate.6,immigrationRate.7,immigrationRate.8,...,immigrationRate.28,immigrationRate.29,immigrationRate.30,immigrationRate.31,immigrationRate.32,immigrationRate.33,immigrationRate.34,immigrationRate.35,immigrationRate.36,immigrationRate.37
0,1500,75000000,1.77617742123068,1.5791666019377057,1.3564593648686327,1.0290318082443455,0.8309495941346164,1.088885174273366,1.268104040761498,0.7289717025010218,...,2.2107324768822356,2.2411181611980076,2.352986094749161,2.534694539986346,2.732905724470472,3.377322971797941,3.210577648831024,3.7414428676570353,4.084673587779051,3.585954066749626
1,1501,75050000,1.5845992041810382,1.8629292634406733,1.3457432385454688,1.2837594867332434,1.6474640646350924,1.9713155766050114,1.5934718073104643,0.9218426495545173,...,2.124690843743514,2.4710071416356527,2.554790225852067,2.8959745085898074,2.644027955945634,3.014394029875279,3.125949563611012,2.9654545188759416,3.2676364552767705,3.0300594464099198
2,1502,75100000,1.1208746621728445,2.043414149397989,1.9608946594743097,1.9351348103168267,2.1004833418048174,1.703796235961993,1.1374870142088451,0.6661549876500423,...,1.6524055405110538,1.8007892792755282,2.1256337575534454,1.997562465016719,2.4736472627594126,2.959733436128652,2.686220802681206,3.668118420926194,3.346730510806702,3.5010149690695536
3,1503,75150000,1.778662678184815,2.1243183123003964,1.5203845919716892,1.189975492986673,1.4564419868305836,1.2266641876671485,1.228323601907395,0.913290618968494,...,1.2533504556654818,1.830786627287511,2.0744002175719456,2.9742499978168606,2.8617560264891893,3.3488127139559927,3.736130123582996,3.8086855273508884,3.874411782270042,3.989422975810455
4,1504,75200000,1.9520549933977693,2.292978270416015,1.9286389414990563,1.4041389789640086,1.9636263745573164,1.7991112210089057,1.2873512411909407,1.5328536971035343,...,2.756768977332072,1.9803704196498781,2.674831599144774,3.1914157423124956,3.355943705250025,3.611227380823505,3.3486060686880332,3.8436920496256906,4.437033821495871,5.155671219782473


In [875]:
percent_df = generate_percent_intro_df(mig_df_f)
percent_df.head()



,intro.percent.1,intro.percent.2,intro.percent.3,intro.percent.4,intro.percent.5,intro.percent.6,intro.percent.7,intro.percent.8,intro.percent.9,intro.percent.10,...,intro.percent.25,intro.percent.26,intro.percent.27,intro.percent.28,intro.percent.29,intro.percent.30,intro.percent.31,intro.percent.32,intro.percent.33,intro.percent.34
0,0.029144,0.156942,-3.340137,0.033855,0.006165,0.021916,0.050348,0.013246,0.034449,0.035671,...,1.918897,0.033327,0.050741,0.026027,0.034943,0.168784,0.033608,0.050823,0.063862,0.041873
1,0.050112,0.281226,1.359526,0.081428,0.018336,0.054743,0.040267,0.022211,0.025211,0.045526,...,0.802637,0.030127,0.019069,0.021984,0.057530,0.071711,0.077744,0.034886,0.019779,0.018239
2,0.014864,0.302786,1.096673,0.039823,0.024262,0.037215,0.042650,0.016517,0.038270,0.047002,...,0.772507,0.077746,0.009565,0.015913,0.036236,0.050855,0.039432,0.028458,0.012980,0.006225
3,0.095573,1.184920,1.072725,0.082767,0.009965,0.030450,0.039858,0.022300,0.059959,0.042838,...,2.140760,0.203330,0.026656,0.020779,0.024145,0.024126,0.096316,0.068987,0.032463,0.036644
4,0.029835,0.647180,2.210094,0.063428,0.017017,0.048766,0.046137,0.033356,0.043844,0.069579,...,0.247418,0.086361,0.055926,0.016787,0.015856,0.045305,0.140097,0.104082,0.027085,0.024209


In [876]:
summary_percent_df = generate_summary_df(percent_df)
summary_percent_df = summary_percent_df.reset_index()



In [877]:
#making sure that any numbers >1 are excluded as outliers from stochastic noise
percent_df =pd.DataFrame(np.where(percent_df <1, percent_df, 1), columns=percent_df.columns )
percent_df =pd.DataFrame(np.where(percent_df >0, percent_df, 0), columns=percent_df.columns )

#format dates
summary_percent_df['days'] = (summary_percent_df.interval.astype(int)-3)*7 #the 1.5 adjustment is made due to the fact that we take the difference of Nes over three weeks 
summary_percent_df['date'] = dt.strptime("2024-09-12",  "%Y-%m-%d") - summary_percent_df.days.map(timedelta)

summary_percent_df.head()





,index,interval,mean_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,days,date
0,0,1,0.028318,2.318153,-2.198683,0.051378,0.000508,-14,2024-09-26
1,0,2,0.070128,2.028746,-1.449467,0.098266,0.001178,-7,2024-09-19
2,0,3,0.111901,6.964512,-5.084110,0.411852,0.002327,0,2024-09-12
3,0,4,0.039584,2.595849,-1.969241,0.053943,0.002569,7,2024-09-05
4,0,5,0.009909,0.033867,0.001432,0.010773,0.003337,14,2024-08-29


In [878]:
columns = ["mean_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
for column in columns:
    # Create a new column for the moving average
    summary_percent_df[f'{column}_MA'] = summary_percent_df[column].rolling(4, min_periods =1).mean()


In [879]:
summary_percent_df.head()

,index,interval,mean_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,days,date,mean_percent_MA,upper_hpd_log_95_MA,lower_hpd_log_95_MA,upper_hpd_log_50_MA,lower_hpd_log_50_MA
0,0,1,0.028318,2.318153,-2.198683,0.051378,0.000508,-14,2024-09-26,0.028318,2.318153,-2.198683,0.051378,0.000508
1,0,2,0.070128,2.028746,-1.449467,0.098266,0.001178,-7,2024-09-19,0.049223,2.173449,-1.824075,0.074822,0.000843
2,0,3,0.111901,6.964512,-5.084110,0.411852,0.002327,0,2024-09-12,0.070116,3.770470,-2.910753,0.187165,0.001338
3,0,4,0.039584,2.595849,-1.969241,0.053943,0.002569,7,2024-09-05,0.062483,3.476815,-2.675375,0.153860,0.001645
4,0,5,0.009909,0.033867,0.001432,0.010773,0.003337,14,2024-08-29,0.057881,2.905743,-2.125347,0.143709,0.002353


In [880]:
line1 = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.9, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month",grid=False,  format="%b %Y")),
    alt.Y('lower_hpd_log_50_MA',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, 1))),
    alt.Y2('upper_hpd_log_50_MA' )
).properties(
    width=850,
    height=300
)

band = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.2, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month",grid=False,  format="%b %Y")),
    alt.Y('lower_hpd_log_95_MA',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, 1))),
    alt.Y2('upper_hpd_log_95_MA' )
).properties(
    width=850,
    height=300
)

#line1
intro_plot = (line1 + band).resolve_scale(y= "shared") ; intro_plot

alt.LayerChart(...)

In [881]:
## now calculate the true percentage of cases due to intros from simulations

In [949]:
simulation_df["whole_time"] = np.ceil(simulation_df.t *366)

In [950]:
daily_sim_cases = simulation_df.groupby(["whole_time"]).agg({'IntroCount':'mean','I':'mean'}).reset_index()


In [951]:
daily_sim_cases["diff_intro"] = daily_sim_cases.IntroCount.diff()
daily_sim_cases["diff_I"] = daily_sim_cases.I.diff()

daily_sim_cases["diff_I"][daily_sim_cases["diff_I"] < 0 ] = 0
daily_sim_cases["diff_intro"][daily_sim_cases["diff_intro"] < 0 ] = 0

In [952]:
counter = 0
daily_sim_cases["week_time"] = np.nan
for index, row in daily_sim_cases.iterrows():
    if row.whole_time %7 == 0:
        daily_sim_cases.loc[index,"week_time"] = counter 
        counter +=1

In [953]:
daily_sim_cases = daily_sim_cases.bfill()

In [954]:
weekly_sim_cases = daily_sim_cases.groupby(["week_time"]).agg({'diff_intro':'mean','I':'mean'}).reset_index()

In [955]:
weekly_sim_cases["percent_intro"] = weekly_sim_cases.diff_intro/(weekly_sim_cases.diff_intro+ weekly_sim_cases.I)

In [956]:
weekly_sim_cases.head()

,week_time,diff_intro,I,percent_intro
0,0.0,1.000000,0.100000,0.909091
1,1.0,1.261905,2.833333,0.308140
2,2.0,0.746499,14.508310,0.048935
3,3.0,0.924632,36.267604,0.024861
4,4.0,0.638393,51.346487,0.012280


In [957]:
columns = ["percent_intro"]
for column in columns:
    # Create a new column for the moving average
    weekly_sim_cases[f'{column}_MA'] = weekly_sim_cases[column].rolling(3, min_periods =1, center = True).mean()

In [958]:
weekly_sim_cases.head()

,week_time,diff_intro,I,percent_intro,percent_intro_MA
0,0.0,1.000000,0.100000,0.909091,0.608615
1,1.0,1.261905,2.833333,0.308140,0.422055
2,2.0,0.746499,14.508310,0.048935,0.127312
3,3.0,0.924632,36.267604,0.024861,0.028692
4,4.0,0.638393,51.346487,0.012280,0.016491


In [959]:
percent_intro_sim = alt.Chart(weekly_sim_cases).mark_line(interpolate='monotone', opacity = 0.8, clip = True, color = "black").encode(
    alt.X('week_time', axis=alt.Axis(title="")),
    alt.Y('percent_intro_MA',axis=alt.Axis(title="percent of cases due to introductions", grid=False)),
    
    
).properties(
    width=850,
    height=300)

percent_intro_sim


alt.Chart(...)

In [893]:
line1 + percent_intro_sim

alt.LayerChart(...)

In [894]:
line1 = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.9, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month",grid=False,  format="%b %Y")),
    alt.Y('lower_hpd_log_50_MA',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, 1))),
    alt.Y2('upper_hpd_log_50_MA' )
).properties(
    width=850,
    height=300
)
line1 +percent_intro_sim

alt.LayerChart(...)